In [1]:
import pandas as pd

In [2]:
df_mapping = pd.read_csv("mrf_ehr_mapping_v1.csv")

In [3]:
df_mrf = pd.read_csv("MRF-SIMBA.csv")

In [4]:
df_datim  = pd.read_csv("DATIM-ONLY-SIMBA.csv")

In [5]:
df_merge = pd.merge(df_datim, df_mapping,  left_on='Facility', right_on='DATIM_Site_Name', how = 'left' )

In [6]:
df_merge.columns

Index(['Facility', 'TX_CURR ', 'MRF_Province', 'MRF_District', 'MRF_Facility',
       'EHR_Province', 'EHR_District', 'DATIM_Site_Name', 'EHR_FacilityName',
       'Facility ID', 'E-HR Status'],
      dtype='object')

In [7]:
df_merge_mrf = pd.merge(df_merge, df_mrf,  left_on='MRF_Facility', right_on='Facility', how = 'left',suffixes=('_datim', '_mrf') )

In [8]:
df_merge_mrf.columns

Index(['Facility_datim', 'TX_CURR _datim', 'MRF_Province', 'MRF_District',
       'MRF_Facility', 'EHR_Province', 'EHR_District', 'DATIM_Site_Name',
       'EHR_FacilityName', 'Facility ID', 'E-HR Status', 'Province',
       'District', 'Facility_mrf', 'TX_CURR _mrf', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13'],
      dtype='object')

In [9]:
df_filtered = df_merge_mrf[[
    'MRF_Province', 'MRF_District',
       'MRF_Facility','Facility_datim','TX_CURR _datim','TX_CURR _mrf'
]]

In [10]:
df_filtered

,MRF_Province,MRF_District,MRF_Facility,Facility_datim,TX_CURR _datim,TX_CURR _mrf
0,Mashonaland West,Makonde,Alaska Munic,Alaska - 100005 - Clinic,561,1028.0
1,Matabeleland South,Insiza,Amazon,Amazon Clinic,409,473.0
2,Harare Metropolitan,Harare City,Arcadia,Arcadia - 100015 - Clinic,603,973.0
3,Harare Metropolitan,Harare City,Avondale,Avondale - 100030 - Clinic,1274,1314.0
4,Mashonaland Central,Guruve,Bakasa,Bakasa - 100035 - Clinic,500,479.0
...,...,...,...,...,...,...
452,NaN,NaN,NaN,Murewa Gvt District Hospital,1612,NaN
453,Harare Metropolitan,Chitungwiza,St Mary's,St. Mary's - 101633 - Clinic,6695,6925.0
454,Harare Metropolitan,Harare City,Highlands Primary Care,Highlands PCC,267,2064.0
455,NaN,NaN,NaN,Hopley,2754,NaN


In [16]:
# 1. Load the reference list of 204 sites
df_reference = pd.read_csv("Facility List and Reporting Sources.csv") 
df_filtered['Facility_datim'] = df_filtered['Facility_datim'].astype(str).str.strip()
df_reference['Facility Name'] = df_reference['Facility Name'].astype(str).str.strip()

# 'how=right' ensures ALL 204 sites in df_reference are kept
df_final_204 = pd.merge(
    df_filtered, 
    df_reference, 
    left_on='Facility_datim', 
    right_on='Facility Name', 
    how='right'
)

# 3. Clean up: If a site didn't match, 'Facility_datim' will be blank. 
# We fill it using the name from the reference list so you can see it.
df_final_204['Facility_datim'] = df_final_204['Facility_datim'].fillna(df_final_204['Facility Name'])

# 4. Export the complete list for mapping
df_final_204.to_csv("MRF - DATIM for complete sites.csv", index=False)

/tmp/ipykernel_20230/2743819938.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Facility_datim'] = df_filtered['Facility_datim'].astype(str).str.strip()


In [17]:
df_filtered.to_csv("MRF - DTAIM for complete sites.csv", index=False)